# Testing the Linear Model trained in TensorFlow 

In [1]:
from __future__ import print_function
from link_bot_notebooks import notebook_finder
from link_bot_notebooks import toy_problem_optimization_common as tpo
from math import acos
import numpy as np
import matplotlib.pyplot as plt
from link_bot_notebooks.linear_tf_model import LinearTFModel

importing Jupyter notebook from /home/pmitrano/catkin_ws/src/link_bot/link_bot_notebooks/src/link_bot_notebooks/toy_problem_optimization_common.ipynb


In [2]:
args = {'checkpoint': "log_data/init_A/December_10_23:11:39__0a0464776e/nn.ckpt-175000", 'log': False, 'batch_size': 1024}
dataset = "../../../link_bot_teleop/data/random_50_steps.txt"
n_steps = 50
dt = 0.1
x = tpo.load_train(dataset, N=6, L=2, n_steps=n_steps, extract_func=tpo.link_pos_vel_extractor2(6))
model = LinearTFModel(args, N=6, M=2, L=2, n_steps=1, dt=dt)
model.load()

INFO:tensorflow:Restoring parameters from log_data/init_A/December_10_23:11:39__0a0464776e/nn.ckpt-175000
Restored ckpt log_data/init_A/December_10_23:11:39__0a0464776e/nn.ckpt-175000 at step 175000


In [ ]:
goal = np.array([[0], [0], [1], [0], [2], [0]])

In [3]:
# A, B, C, D, c_loss, sp_loss, cp_loss, reg, loss = model.evaluate(x, goal, display=True)

In [8]:
A = model.get_A()
print(model)

A:
[[ 0.19 -0.44  0.05  0.01 -0.01 -0.09]
 [-0.25 -0.1  -0.02 -0.04 -0.09 -0.04]]
B:
[[-0.03  0.15]
 [-0.11  0.01]]
C:
[[  7.51 -26.44]
 [-23.28  -7.18]]
D:
[[ 3.13 -0.44]
 [-0.19  5.5 ]]



In [12]:
s0 = np.array([[0], [0], [1], [0], [2], [0]])
print(s0.T)
s1 = np.linalg.lstsq(A, model.predict_from_s(s=s0, u=np.array([[[0], [0]]])), rcond=None)[0]
print(s1.T)

[[0 0 1 0 2 0]]
[[0.63 0.26 0.06 0.11 0.23 0.1 ]]


In [ ]:
data = tpo.load_data(dataset, g=goal, extract_func=tpo.two_link_pos_vel_extractor)

In [ ]:
plt.figure(figsize=(12,12))
m = 7
c = 0
res = 1
tpo.plot_costmap_2(plt, model, data, goal, resolution=res, minimum=c-m, maximum=c+m)
ax = plt.gca()
tpo.policy_quiver(model, goal, ax, c, c, res, m, scale=25)
plt.xlabel("X (m)")
plt.ylabel("Y (m)");

In [ ]:
points = np.array([[1,0], [1,1], [0,1], [-1,1], [-1,0], [-1,-1], [0,-1], [1,-1]])
R = model.get_A()[:,:2]
new_points = []
for p in points:
    new_p = np.dot(R, p)
    new_points.append(new_p)
new_points = np.array(new_points)
    
plt.figure(figsize=(10,10))
plt.scatter([points[0,0]], [points[0,1]], s=150)
plt.scatter([new_points[0,0]], [new_points[0,1]], s=150)
plt.scatter([p[0] for p in points], [p[1] for p in points])
plt.scatter([p[0] for p in new_points], [p[1] for p in new_points])
plt.plot([p[0] for p in points], [p[1] for p in points], label='original')
plt.plot([p[0] for p in new_points], [p[1] for p in new_points], label='latent')
plt.legend()
plt.axis("equal");